In [2]:
import tabula
import numpy as np
import csv

## Obtain PDF

file updates daily: http://stopcoronavirus.mcgm.gov.in/assets/docs/Dashboard.pdf

## Scrape PDF

Obtain tables from file. Convert and save tables as csv files

* using tablua for scraping: http://theautomatic.net/2019/05/24/3-ways-to-scrape-tables-from-pdfs-with-python/

* https://nbviewer.jupyter.org/github/chezou/tabula-py/blob/master/examples/tabula_example.ipynb

In [3]:
file = "http://stopcoronavirus.mcgm.gov.in/assets/docs/Dashboard.pdf"

### Ward-wise new cases

Positive cases for the last 7 days 

In [4]:
new_case_growth = tabula.read_pdf(file, pages=24, multiple_tables=False)

new_cases = new_case_growth[0]

In [5]:
nc_expected_header = ['Date of report', 'RC', 'HW', 'RS', 'RN', 'PS', 'A', 'C', 'D', 'KW', 'T', 'PN', 'N', 'FN', 'FS', 'MW', 'ME', 'B', 'E', 'GS', 'KE', 'GN', 'S', 'HE', 'L', 'Grand Total']


In [6]:
# check that correct table was scraped

# check headers
if (new_cases.columns == nc_expected_header).all():
    pass
else:
    print('Incorrect columns in ward cases table')
    
    
# check rows
row_11_values = new_cases.loc[11, nc_expected_header[1:]]

if row_11_values.isnull().all(): # value for each ward should be null
    pass
else:
    print('Unexpected values in row 11')
    
for row in range(1,11):
    if np.array_equal(new_cases.loc[row, nc_expected_header[1:-1]], new_cases.loc[row, nc_expected_header[1:-1]].astype(str)): # value for each ward should be a str
        pass
    else:
        print('Unexpected values in rows 1-10')

In [7]:
new_cases.drop(labels=11, inplace=True)

### Save ward identifier with corresponding ward name in a dictionary

In [8]:
wards = new_cases.iloc[0][1:] # series of ward names, index is identifiers
identifiers = new_cases.iloc[0][1:].index
ward_identifiers = {}

In [9]:
wards.drop('Grand Total', inplace=True)

wards

RC           Borivali
HW             Bandra
RS          Kandivali
RN            Dahisar
PS           Goregaon
A              Colaba
C       Marine\rLines
D         Grant\rRoad
KW         Andheri\rW
T              Mulund
PN              Malad
N           Ghatkopar
FN            Matunga
FS              Parel
MW         Chembur\rW
ME         Chembur\rE
B     Sandhurst\rRoad
E             Byculla
GS        Elphinstone
KE         Andheri\rE
GN              Dadar
S             Bhandup
HE               Khar
L               Kurla
Name: 0, dtype: object

In [10]:
for ward in wards:
    text = str(ward)
    name = text.replace('\r',' ')
    index = wards[wards==text].index[0]
    ward_identifiers[index] = name

In [11]:
ward_identifiers

{'RC': 'Borivali',
 'HW': 'Bandra',
 'RS': 'Kandivali',
 'RN': 'Dahisar',
 'PS': 'Goregaon',
 'A': 'Colaba',
 'C': 'Marine Lines',
 'D': 'Grant Road',
 'KW': 'Andheri W',
 'T': 'Mulund',
 'PN': 'Malad',
 'N': 'Ghatkopar',
 'FN': 'Matunga',
 'FS': 'Parel',
 'MW': 'Chembur W',
 'ME': 'Chembur E',
 'B': 'Sandhurst Road',
 'E': 'Byculla',
 'GS': 'Elphinstone',
 'KE': 'Andheri E',
 'GN': 'Dadar',
 'S': 'Bhandup',
 'HE': 'Khar',
 'L': 'Kurla'}

In [12]:
len(ward_identifiers) == len(wards) # check that dictionary has all wards

True

In [13]:
new_cases.drop(labels=0, inplace=True)

new_cases

Date of report    RC    HW    RS    RN    PS     A     C     D    KW  ...  \
1   As on 22nd Aug  7331  3752  6473  3524  4507  3322  1892  5671  7539  ...   
2           23-Aug    77    50    59    24    50    43    25    45    35  ...   
3           24-Aug    61    18    62    21    34    17    13    32    70  ...   
4           25-Aug    38    14    60    25    23    11     9    40    29  ...   
5           26-Aug   178    82   129    83    73    70    23    86   131  ...   
6           27-Aug   161    54   130    59    57    37    16    82    80  ...   
7           28-Aug   128    56    76    46    37    33    22    58    68  ...   
8           29-Aug   136    76    75    51    69    37    33    61    65  ...   
9   As on 29th Aug  8110  4102  7064  3833  4850  3570  2033  6075  8017  ...   
10  Days to double    48    54    56    58    66    67    68    71    79  ...   

      ME     B     E    GS    KE    GN     S    HE     L Grand Total  
1   4657  1190  5365  5823  8341  7895  7331  4607  5803    131301.0  
2     20     5    30    30    29    31    40    29    28       919.0  
3     30     7    25    29    35    33    12    13    15       688.0  
4     16     4    20    11    15    17    25    21    17       537.0  
5     59     6    57    44    67    72    67    30    32      1671.0  
6     24     7    34    43    59    48    43    26    34      1295.0  
7     35    13    19    49    56    60    50    23    35      1180.0  
8     28     9    43    40    79    55    53    27    41      1339.0  
9   4869  1241  5593  6069  8681  8211  7621  4776  6005    138930.0  
10   109   116   117   117   121   124   125   135   142        86.0  

[10 rows x 26 columns]

### Proactive screening of elderly to protect them and contain mortality

In [14]:
elderly_screening = tabula.read_pdf(file, pages=21, multiple_tables=False)

In [15]:
elderly = elderly_screening[0]

In [16]:
e_expected_header = ['Wards', 'Total No. of Houses', 'Population', 'Total No. of', 'Sr Citizen', '* Sr Citizen', 'Unnamed: 6', 'Unnamed: 7']

In [17]:
elderly

Wards Total No. of Houses Population     Total No. of Sr Citizen  \
0       NaN            Surveyed    Covered  Senior Citizens    SPO2>95   
1     Total                 NaN        NaN              NaN        NaN   
2   (Daily)              45,036    166,721            7,020      7,016   
3       H/W                 643      2,346               96         95   
4       M/E                 608      2,404              258        257   
5         N               2,586     10,062            1,110      1,109   
6         T                 979      3,206              384        383   
7         A                 425      1,629               15         15   
8         B                 352      1,468               43         43   
9         C                 801      2,467               49         49   
10        D                 393      1,618              120        120   
11        E               1,617      6,765              176        176   
12      F/S               1,129      4,644              150        150   
13      F/N                 424      1,506              251        251   
14      G/S               3,311     13,322              333        333   
15      G/N               2,429      8,465              150        150   
16      H/E                 500      2,153               59         59   
17      K/E               2,946     10,681              607        607   
18      K/W                 543      2,229               32         32   
19      P/S                 833      2,689              133        133   
20      P/N               3,437     12,102              257        257   
21      R/S               4,803     17,690              191        191   
22      R/C                 862      3,069               50         50   
23      R/N               8,353     27,950              317        317   
24        L               4,547     19,337            1,865      1,865   
25      M/W               1,001      3,256              224        224   
26        S               1,514      5,663              150        150   

   * Sr Citizen  Unnamed: 6                          Unnamed: 7  
0       SPO2<95         NaN                   Summary till date  
1           NaN         NaN                                 NaN  
2           NaN         4.0                                 NaN  
3           NaN         1.0                                 NaN  
4           NaN         1.0                         1,72,25,210  
5           NaN         1.0                                 NaN  
6           NaN         1.0  Total Population covered till date  
7           NaN         0.0                                 NaN  
8           NaN         0.0                                 NaN  
9           NaN         0.0                            9,08,503  
10          NaN         0.0                                 NaN  
11          NaN         0.0                                 NaN  
12          NaN         0.0   Senior Citizens covered till date  
13          NaN         0.0                                 NaN  
14          NaN         0.0                                 NaN  
15          NaN         0.0                                 NaN  
16          NaN         0.0                            9,05,391  
17          NaN         0.0                                 NaN  
18          NaN         0.0                                 NaN  
19          NaN         0.0        Senior Citizens with SPO2>95  
20          NaN         0.0                           till date  
21          NaN         0.0                                 NaN  
22          NaN         0.0                               3,112  
23          NaN         0.0                                 NaN  
24          NaN         0.0                                 NaN  
25          NaN         0.0        Senior Citizens with SPO2<95  
26          NaN         0.0                                  21

In [18]:
# check that correct table was scraped

# check headers
if (elderly.columns == e_expected_header).all():
    pass
else:
    print('Incorrect columns in elderly table')
    
    
# check rows
row_1_values = elderly.loc[1, e_expected_header[1:]]

if row_1_values.isnull().all(): # value for each ward should be null
    pass
else:
    print('Unexpected values in row 1')
    
    
# check columns
for column in e_expected_header[:4]:
    if np.array_equal(elderly[column][2:], elderly[column][2:].astype(str)):
        pass
    else:
        print('Unexpected values in ' + str(column))
    
if np.array_equal(elderly['Unnamed: 6'][2:], elderly['Unnamed: 6'][2:].astype(float)):
    pass
else:
    print('Unexpected values in ' + str(column))

In [19]:
elderly.loc[2, 'Wards'] = 'Daily Totals'

In [20]:
elderly.drop(labels=[0,1], inplace=True)

In [21]:
elderly.drop(columns=['* Sr Citizen','Unnamed: 7'], inplace=True)

In [22]:
elderly.columns = [
    'Wards', 'Total No. of Houses Surveyed', 'Population Covered', 'Total No. of Senior Citizens', 'Sr Citizen SPO2>95','Sr Citizen SPO2<95'
]

In [23]:
elderly.index = elderly['Wards']
elderly.drop(columns=['Wards'], inplace=True)

In [24]:
elderly

Total No. of Houses Surveyed Population Covered  \
Wards                                                          
Daily Totals                       45,036            166,721   
H/W                                   643              2,346   
M/E                                   608              2,404   
N                                   2,586             10,062   
T                                     979              3,206   
A                                     425              1,629   
B                                     352              1,468   
C                                     801              2,467   
D                                     393              1,618   
E                                   1,617              6,765   
F/S                                 1,129              4,644   
F/N                                   424              1,506   
G/S                                 3,311             13,322   
G/N                                 2,429              8,465   
H/E                                   500              2,153   
K/E                                 2,946             10,681   
K/W                                   543              2,229   
P/S                                   833              2,689   
P/N                                 3,437             12,102   
R/S                                 4,803             17,690   
R/C                                   862              3,069   
R/N                                 8,353             27,950   
L                                   4,547             19,337   
M/W                                 1,001              3,256   
S                                   1,514              5,663   

             Total No. of Senior Citizens Sr Citizen SPO2>95  \
Wards                                                          
Daily Totals                        7,020              7,016   
H/W                                    96                 95   
M/E                                   258                257   
N                                   1,110              1,109   
T                                     384                383   
A                                      15                 15   
B                                      43                 43   
C                                      49                 49   
D                                     120                120   
E                                     176                176   
F/S                                   150                150   
F/N                                   251                251   
G/S                                   333                333   
G/N                                   150                150   
H/E                                    59                 59   
K/E                                   607                607   
K/W                                    32                 32   
P/S                                   133                133   
P/N                                   257                257   
R/S                                   191                191   
R/C                                    50                 50   
R/N                                   317                317   
L                                   1,865              1,865   
M/W                                   224                224   
S                                     150                150   

              Sr Citizen SPO2<95  
Wards                             
Daily Totals                 4.0  
H/W                          1.0  
M/E                          1.0  
N                            1.0  
T                            1.0  
A                            0.0  
B                            0.0  
C                            0.0  
D                            0.0  
E                            0.0  
F/S                          0.0  
F/N                          0.0  
G/S                          0.0  
G/N                          0

**final step: save scraped data**

In [ ]:
saved_files_path = '/Users/wasilaq/SWB/data-analysis/' # location where csv files should be saved

new_cases.to_csv(saved_files_path + 'new_cases.csv')
elderly.to_csv(saved_files_path + 'elderly.csv')


# save ward identifiers dictionary and keep in Github, for reference
with open('ward_identifiers.csv', 'w') as f:
    w = csv.DictWriter(f, ward_identifiers.keys())
    w.writeheader()
    w.writerow(ward_identifiers)